In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
import matplotlib as mpl
import copy

In [4]:
disease_genes_read= "/home/andre/Documents/trabalho/datasets/Disease_genes/genehancer/GeneHancer_v5.20.gff"
dg_df = pd.read_csv(disease_genes_read, sep="\t")

dg_df

,#chrom,source,feature name,start,end,score,strand,frame,attributes
0,chr20,GeneHancer,Enhancer,208888,209430,0.76,.,.,genehancer_id=GH20J000209;connected_gene=TBC1D...
1,chr20,GeneHancer,Promoter/Enhancer,226801,228800,0.71,.,.,genehancer_id=GH20J000227;connected_gene=DEFB1...
2,chr20,GeneHancer,Promoter/Enhancer,266200,267649,1.72,.,.,genehancer_id=GH20J000266;connected_gene=ENSG0...
3,chr20,GeneHancer,Promoter/Enhancer,279140,279609,0.51,.,.,genehancer_id=GH20J000280;connected_gene=C20or...
4,chr20,GeneHancer,Promoter/Enhancer,289400,292542,2.12,.,.,genehancer_id=GH20J000289;connected_gene=C20or...
...,...,...,...,...,...,...,...,...,...
419015,chr19,GeneHancer,Enhancer,49645262,49646700,0.14,.,.,genehancer_id=GH19J049645;connected_gene=HSALN...
419016,chr17,GeneHancer,Enhancer,17222392,17224411,0.26,.,.,genehancer_id=GH17J017222;connected_gene=FLCN;...
419017,chr17,GeneHancer,Enhancer,17220947,17221921,0.14,.,.,genehancer_id=GH17J017220;connected_gene=NT5M;...
419018,chr17,GeneHancer,Enhancer,17215652,17216481,0.26,.,.,genehancer_id=GH17J017216;connected_gene=FLCN;...


O Dataset consiste em promotores e enhancers relacionados com genes de doenças.

In [5]:
dg_df["feature name"].unique()

array(['Enhancer', 'Promoter/Enhancer', 'Promoter'], dtype=object)

Feature name apenas tem os valores: 'Enhancer', 'Promoter/Enhancer', 'Promoter'. Portanto não há gene regions.

In [7]:
disease_genes_read2= "/home/andre/Documents/trabalho/datasets/Disease_genes/genehancer/GeneHancer_v5.20.bed"
dg_df2 = pd.read_csv(disease_genes_read2, sep="\t", header=None)

dg_df2

,0,1,2,3
0,chr20,208887,209430,GH20J000209
1,chr20,226800,228800,GH20J000227
2,chr20,266199,267649,GH20J000266
3,chr20,279139,279609,GH20J000280
4,chr20,289399,292542,GH20J000289
...,...,...,...,...
419015,chr19,49645261,49646700,GH19J049645
419016,chr17,17222391,17224411,GH17J017222
419017,chr17,17220946,17221921,GH17J017220
419018,chr17,17215651,17216481,GH17J017216


A diferença nos indices entre os ficheiros .gff e .bed é que o {start em gff = ao indice inicial em bed +1}
porque os indices em bed começam em 0 e em gff começam em 1.

## Tratamento dos dados do ficheiro .gff

In [8]:
dg_df

,#chrom,source,feature name,start,end,score,strand,frame,attributes
0,chr20,GeneHancer,Enhancer,208888,209430,0.76,.,.,genehancer_id=GH20J000209;connected_gene=TBC1D...
1,chr20,GeneHancer,Promoter/Enhancer,226801,228800,0.71,.,.,genehancer_id=GH20J000227;connected_gene=DEFB1...
2,chr20,GeneHancer,Promoter/Enhancer,266200,267649,1.72,.,.,genehancer_id=GH20J000266;connected_gene=ENSG0...
3,chr20,GeneHancer,Promoter/Enhancer,279140,279609,0.51,.,.,genehancer_id=GH20J000280;connected_gene=C20or...
4,chr20,GeneHancer,Promoter/Enhancer,289400,292542,2.12,.,.,genehancer_id=GH20J000289;connected_gene=C20or...
...,...,...,...,...,...,...,...,...,...
419015,chr19,GeneHancer,Enhancer,49645262,49646700,0.14,.,.,genehancer_id=GH19J049645;connected_gene=HSALN...
419016,chr17,GeneHancer,Enhancer,17222392,17224411,0.26,.,.,genehancer_id=GH17J017222;connected_gene=FLCN;...
419017,chr17,GeneHancer,Enhancer,17220947,17221921,0.14,.,.,genehancer_id=GH17J017220;connected_gene=NT5M;...
419018,chr17,GeneHancer,Enhancer,17215652,17216481,0.26,.,.,genehancer_id=GH17J017216;connected_gene=FLCN;...


In [9]:
dg_df["attributes"][0]

'genehancer_id=GH20J000209;connected_gene=TBC1D20;score=12.42;connected_gene=piR-34165;score=0.45;connected_gene=DEFB128;score=0.37;connected_gene=DEFB129;score=0.37;connected_gene=HSALNG0128009;score=0.27'

Cada região Enhancer ou Promoter tem vários genes mais ou menos relacionados (score). Essa lista de genes na coluna "atributes" contem para cada gene apenas o seu simbolo e o seu score (relação com a região). Portanto não contem o seu ID que seria o ideal.
Para além disso os genes de simbolos "piR-34165" (só de ratos) e "HSALNG0128009" não foram encontrados na base de dados da NIH.

In [10]:
dg_df["attributes"][:]

0         genehancer_id=GH20J000209;connected_gene=TBC1D...
1         genehancer_id=GH20J000227;connected_gene=DEFB1...
2         genehancer_id=GH20J000266;connected_gene=ENSG0...
3         genehancer_id=GH20J000280;connected_gene=C20or...
4         genehancer_id=GH20J000289;connected_gene=C20or...
                                ...                        
419015    genehancer_id=GH19J049645;connected_gene=HSALN...
419016    genehancer_id=GH17J017222;connected_gene=FLCN;...
419017    genehancer_id=GH17J017220;connected_gene=NT5M;...
419018    genehancer_id=GH17J017216;connected_gene=FLCN;...
419019    genehancer_id=GH20J064315;connected_gene=HSALN...
Name: attributes, Length: 419020, dtype: object

In [11]:
genes_list = []
for row in dg_df["attributes"]:
    row_elements = row.split(";")
    for i in range (1, len(row_elements)-1, 2):
        gene_symbol = row_elements[i].split("=")[1]
        genes_list.append(gene_symbol)

len(genes_list)

2552245

In [12]:
df = pd.DataFrame(genes_list)
df

,0
0,TBC1D20
1,piR-34165
2,DEFB128
3,DEFB129
4,HSALNG0128009
...,...
2552240,HSALNG0131740
2552241,piR-39425
2552242,HSALNG0131742
2552243,HSALNG0131741


In [13]:
disease_genes_list = df[0].unique()
print(disease_genes_list, len(disease_genes_list))

['TBC1D20' 'piR-34165' 'DEFB128' ... 'HSALNG0131740' 'piR-39425'
 'HSALNG0131742'] 265951


### .dff data treatment

In [14]:
def subtract_start(dataframe):
    new_row = []
    for row in dataframe['start']:
        new_row.append(row - 1)
    
    dataframe.drop('start', axis = 1, inplace=True)

    dataframe.insert(3, "start", new_row, True)

def addCitobandTo_dff(dff_df, cb_df):
    """Adds the citoband column to dff file
    """
    citobands = []
    for row in range(len(dff_df.values)):
        citoband_found = False
        f_index = dff_df["start"][row]
        l_index = dff_df["end"][row]
        chr = dff_df["#chrom"][row]
        for citoband in cb_df.values:
            #print(citoband[0])
            if citoband[0] == chr:
                if int(citoband[1]) < int(f_index) and int(citoband[2])-1 > int(l_index) and not citoband_found:
                    citobands.append(citoband[3])
                    citoband_found = True
        if not citoband_found:
            for citoband_index in range (len(cb_df.values) - 1):
                citoband = cb_df.values[citoband_index]
                next_citoband = cb_df.values[citoband_index + 1]
                if citoband[0] == chr and next_citoband[0] == chr:
                    if int(citoband[1]) < int(f_index) and int(next_citoband[2])-1 > int(l_index) and not citoband_found:
                        citobands.append(citoband[3] + " & " + next_citoband[3])
                        citoband_found = True
            if not citoband_found:
                    citobands.append("cytoband not found")
    
    dff_df.insert(len(dff_df.columns), "cytoband", citobands, True)


def calc_size1(dataframe):
    """Calculate size of region by subtractong end with start
    """
    sizes = []
    for row in dataframe.values:
        size = int(row[4]) - int(row[3])
        sizes.append(size)
    dataframe.insert(len(dataframe.columns), "size", sizes, True)


def writeNewtxt(file_path, df):
    """Writes a pandas dataframe into txt, columns separated by tabs ("\t")
    """
    f = open(file_path, 'w')
    str_cols = ""
    for col in df.columns[:-1]:
        str_cols += col + "\t"
    str_cols += df.columns.values[-1]+ "\n"
    f.write(str(str_cols))
    for row in df.values:
        str_row = ""
        for col in row[:-1]:
            str_row += str(col) + "\t"
        str_row += str(row[-1])+ "\n"
        f.write(str_row)
    f.close



Running code

In [18]:
CitobandPath = "/home/andre/Documents/trabalho/citobands/cytobandFiltered.txt"
#disease_genes_read= "/home/andre/Documents/trabalho/datasets/Disease_genes/genehancer/GeneHancer_v5.20.gff"
dffPath_write = "/home/andre/Documents/trabalho/datasets/Disease_genes/genehancer/GeneHancer_v5.20_processed.gff"

dg_df = pd.read_csv(disease_genes_read, sep="\t")
citobands_df = pd.read_csv(CitobandPath, sep="\t", header=None)


In [48]:
subtract_start(dg_df)

In [49]:
calc_size1(dg_df)

In [50]:
addCitobandTo_dff(dg_df, citobands_df)

In [51]:
dg_df

,#chrom,source,feature name,start,end,score,strand,frame,attributes,size,cytoband
0,chr20,GeneHancer,Enhancer,208887,209430,0.76,.,.,genehancer_id=GH20J000209;connected_gene=TBC1D...,543,p13
1,chr20,GeneHancer,Promoter/Enhancer,226800,228800,0.71,.,.,genehancer_id=GH20J000227;connected_gene=DEFB1...,2000,p13
2,chr20,GeneHancer,Promoter/Enhancer,266199,267649,1.72,.,.,genehancer_id=GH20J000266;connected_gene=ENSG0...,1450,p13
3,chr20,GeneHancer,Promoter/Enhancer,279139,279609,0.51,.,.,genehancer_id=GH20J000280;connected_gene=C20or...,470,p13
4,chr20,GeneHancer,Promoter/Enhancer,289399,292542,2.12,.,.,genehancer_id=GH20J000289;connected_gene=C20or...,3143,p13
...,...,...,...,...,...,...,...,...,...,...,...
419015,chr19,GeneHancer,Enhancer,49645261,49646700,0.14,.,.,genehancer_id=GH19J049645;connected_gene=HSALN...,1439,q13.33
419016,chr17,GeneHancer,Enhancer,17222391,17224411,0.26,.,.,genehancer_id=GH17J017222;connected_gene=FLCN;...,2020,p11.2
419017,chr17,GeneHancer,Enhancer,17220946,17221921,0.14,.,.,genehancer_id=GH17J017220;connected_gene=NT5M;...,975,p11.2
419018,chr17,GeneHancer,Enhancer,17215651,17216481,0.26,.,.,genehancer_id=GH17J017216;connected_gene=FLCN;...,830,p11.2


In [52]:
writeNewtxt(dffPath_write, dg_df)